# visualiziong mpl

In [1]:
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
%matplotlib inline  
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../MNIST_data/', one_hot=True)

train_imgs = mnist.train.images
train_labels = mnist.train.labels
test_imgs = mnist.test.images
test_labels = mnist.test.labels
print("package load")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
package load


## define network

In [2]:
# parameter
learning_rate = 0.001
training_epochs = 10
batch_size = 100
display_step = 2

# Network Parameters
n_input    = 784 # MNIST data input (img shape: 28*28)
n_hidden_1 = 256 # 1st layer num features
n_hidden_2 = 256 # 2nd layer num features
n_hidden_3 = 256 # 3rd layer num features
n_hidden_4 = 256 # 4th layer num features
n_classes  = 10 # MNIST total classes (0-9 digits)

# Store layers weight & bias
stddev = 0.1
with tf.variable_scope("MLP_WEIGHTS"):
    weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
        'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], stddev=stddev)),
        'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4], stddev=stddev)),
        'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes], stddev=stddev))
    }
with tf.variable_scope("MLP_BIASES"):
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'b3': tf.Variable(tf.random_normal([n_hidden_3])),
        'b4': tf.Variable(tf.random_normal([n_hidden_4])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# create model
def multilayer_perception(_x, _weights, _biases, _keep_prob):
    with tf.variable_scope("MLP_LAYER_1"):
        layer_1 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(_x, _weights['h1']), _biases['b1'])), _keep_prob)
    with tf.variable_scope("MLP_LAYER_2"):
        layer_2 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])), _keep_prob)
    with tf.variable_scope("MLP_LAYER_3"):
        layer_3 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer_2, _weights['h3']), _biases['b3'])), _keep_prob)
    with tf.variable_scope("MLP_LAYER_4"):
        layer_4 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer_3, _weights['h4']), _biases['b4'])), _keep_prob)
    
    return (tf.matmul(layer_4, _weights['out']) + biases['out'])
print("network ready")

network ready


## define function

In [4]:
# tf graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
keep_prob = tf.placeholder("float")

# construct model
pred = multilayer_perception(x, weights, biases, keep_prob)

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# accurancy
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accur = tf.reduce_mean(tf.cast(corr, "float"))

# init
init = tf.global_variables_initializer()
print("network ready")

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

network ready


## summary

In [7]:
# run
sess = tf.Session()
sess.run(init)

# summary writer
tf.summary.scalar('cross entropy', loss)
tf.summary.scalar('accuracy', accur)
merged = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter('/tmp/tf_logs/mlp_mnist', graph=sess.graph)

print("summary ready")

INFO:tensorflow:Summary name cross entropy is illegal; using cross_entropy instead.
summary ready


## run